In [1]:
from pathlib import Path; import os
cwd_path = Path.cwd(); set_path = str(cwd_path.parent); os.chdir(set_path)

In [2]:
from Utils.load_binarized_mnist import x_train, x_test, train_ds, test_ds, image_dim
import tensorflow as tf
import numpy as np

In [4]:
from tensorflow.keras.layers import Layer, Conv2D, BatchNormalization, Conv2DTranspose
from tensorflow_addons.layers import WeightNormalization

# Weight Norm

In [9]:
conv = Conv2D(filters=10, kernel_size=(3,3), strides=(1,1))

In [10]:
conv(x_train[0:10, :, :, :]).shape

TensorShape([10, 26, 26, 10])

In [15]:
norm = lambda x: WeightNormalization(x, data_init=True)

In [18]:
conv = norm(Conv2D(filters=10, kernel_size=(3,3), strides=(1,1)))

In [19]:
conv(x_train[0:10, :, :, :]).shape

TensorShape([10, 26, 26, 10])

In [21]:
conv2 = norm(Conv2D(filters=32, kernel_size=(3,3), strides=(2,2)))
conv2(x_train[0:10, :, :, :]).shape

TensorShape([10, 13, 13, 32])

In [ ]:
from Inverse_Autoregressive_Flow_no_reg.Encoder import IAF_Encoder

### transpose conv

In [42]:
from tensorflow_probability import layers

In [43]:
layers.weight_norm.WeightNorm

tensorflow_probability.python.layers.weight_norm.WeightNorm

In [44]:
#conv = norm(Conv2DTranspose(filters=10, kernel_size=(3,3), strides=(1,1)))
conv = layers.weight_norm.WeightNorm(Conv2DTranspose(filters=10, kernel_size=(3,3), strides=(1,1)))

In [45]:
conv(x_train[0:10, :, :, :]).shape

TensorShape([10, 30, 30, 10])

# padding = same vs valid

In [25]:
dense = tf.keras.layers.Dense(2)

In [26]:
x = np.array([[1,1, 1]])

In [27]:
dense(x)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.9913081, -1.8203226]], dtype=float32)>

In [28]:
dense.kernel

<tf.Variable 'dense_3/kernel:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.9124942 , -0.7192813 ],
       [ 0.9132912 , -0.6430105 ],
       [-0.8344773 , -0.45803082]], dtype=float32)>

In [20]:
dense.weights

[<tf.Variable 'dense_2/kernel:0' shape=(2, 2) dtype=float32, numpy=
 array([[-0.75537544, -0.0737685 ],
        [ 0.8396195 , -0.40857232]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [19]:
dir(dense)

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_cast_single_input',
 '_checkpoint_dependencies',
 '_clear_losses',
 '_compute_dtype',
 '_compute_dtype_object',
 '_dedup_weights',
 '_default_training_arg',
 '_deferred_dependencies',
 '_dtype',
 '_dtype_policy',


In [18]:
dense.bias

<tf.Variable 'dense_2/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>

In [16]:
dense.trainable_variables

[<tf.Variable 'dense_2/kernel:0' shape=(2, 2) dtype=float32, numpy=
 array([[-0.75537544, -0.0737685 ],
        [ 0.8396195 , -0.40857232]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [6]:
conv = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', strides=(2, 2), 
                       input_shape=image_dim, padding="same")
conv_out = conv(x_train[0:10])
conv_out.shape

TensorShape([10, 14, 14, 32])

In [18]:
conv = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', strides=(2, 2), 
                       input_shape=image_dim, padding="valid")
conv2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', strides=(2, 2), 
                       input_shape=image_dim, padding="valid")
conv3 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', strides=(2, 2), 
                       input_shape=image_dim, padding="valid")
conv_out = conv(x_train[0:10])
print(conv_out.shape)
conv_out = conv2(conv_out)
print(conv_out.shape)
conv_out = conv3(conv_out)
print(conv_out.shape)

conv_out.shape

(10, 13, 13, 32)
(10, 6, 6, 32)
(10, 2, 2, 32)


TensorShape([10, 2, 2, 32])

In [16]:
deconv = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding="valid")
deconv1 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding="valid")
deconv2 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding="valid")

In [19]:
deconv(conv_out)

<tf.Tensor: shape=(10, 5, 5, 32), dtype=float32, numpy=
array([[[[ 5.93850564e-04, -1.91642274e-03, -3.00803082e-03, ...,
          -1.92411919e-03,  2.53479066e-03,  6.67284941e-04],
         [-1.63979398e-03, -2.12623345e-04, -2.16966565e-03, ...,
           3.13750375e-03,  1.02140917e-03, -2.53997766e-03],
         [ 3.86243337e-03, -1.88302388e-03, -3.96666117e-03, ...,
          -1.07573494e-02,  9.86023527e-03, -3.72777833e-03],
         [ 1.00148004e-02,  3.52762733e-03,  4.73284209e-03, ...,
           2.71352399e-02,  1.91282220e-02, -1.74228363e-02],
         [ 4.32376424e-03,  6.73064590e-03,  4.76729870e-03, ...,
           4.87983134e-03, -1.00358985e-02, -8.08366761e-03]],

        [[-1.99023169e-03, -7.20379641e-04,  2.09826184e-03, ...,
          -3.65595426e-03,  2.11319420e-03, -1.21493556e-03],
         [-8.57212639e-04,  7.50195293e-04, -1.22002803e-03, ...,
          -8.05182557e-04, -2.08987319e-03, -3.71893584e-05],
         [ 1.23140123e-02, -9.16633848e-03,  6

# Identity mapping

In [3]:
conv = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', strides=(2, 2), 
                       input_shape=image_dim, padding="same")

In [4]:
conv_out = conv(x_train[0:10])
conv_out.shape

TensorShape([10, 14, 14, 32])

In [5]:
identity_mapping =  tf.keras.layers.Conv2D(filters=32, kernel_size=(1), padding="same", strides=(2, 2))

In [6]:
id_out = identity_mapping(x_train[0:10])
id_out.shape

TensorShape([10, 14, 14, 32])

In [8]:
#conv_out + id_out

## Identity maping with strides 1

In [9]:
identity_mapping =  tf.keras.layers.Conv2D(filters=32, kernel_size=(1), padding="same", strides=(1,1))

In [10]:
id_out = identity_mapping(x_train[0:10])
id_out.shape

TensorShape([10, 28, 28, 32])

In [20]:
import numpy as np

In [21]:
np.sum(x_train[0:10] != id_out)

30496

In [25]:
np.sum(x_train[0:10] == id_out)

220384

# conv transpose
so strides controls the dimension

In [8]:
conv_trans = tf.keras.layers.Conv2DTranspose

In [9]:
import tensorflow as tf
import numpy as np

In [10]:
latent_dim=3

In [11]:
decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        ]
    )

In [12]:
x_train[0].shape

(28, 28, 1)

In [13]:
x = np.array([[1,2,3]]).astype("float32")

# Method 2
https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE

In [20]:
x2 = tf.keras.layers.Reshape(target_shape=(1, 1, 3))(x)
x2

<tf.Tensor: shape=(1, 1, 1, 3), dtype=float32, numpy=array([[[[1., 2., 3.]]]], dtype=float32)>

In [48]:
test2 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=7, strides=1,
                         padding='valid')(x2)
test2.shape

TensorShape([1, 7, 7, 32])

In [49]:
test3 = tf.keras.layers.Conv2DTranspose(32, 2, strides=2,
                         padding='same')(test2)
test3.shape

TensorShape([1, 14, 14, 32])

# Method 1
https://www.tensorflow.org/tutorials/generative/cvae

In [33]:
y = tf.keras.layers.InputLayer(input_shape=(latent_dim,))(x)
y = tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu)(y)
y = tf.keras.layers.Reshape(target_shape=(7, 7, 32))(y)
y.shape

TensorShape([1, 7, 7, 32])

In [34]:
y2 = tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu')(y)
y2.shape

TensorShape([1, 14, 14, 64])

In [60]:
y3 = tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu')(y2)
y3.shape

TensorShape([1, 28, 28, 32])

In [61]:
y4 = tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same')(y3)
y4.shape

TensorShape([1, 28, 28, 1])

In [63]:
image_dim

(28, 28, 1)